In [1]:
import requests
from bs4 import BeautifulSoup
from IPython.core.display import HTML
from fastcore.all import *
import pandas as pd

In [2]:
main_url = 'https://help.dyalog.com/18.2/Content/Language/Introduction/Language%20Elements.htm?tocpath=Language%20Reference%20Guide%7CSymbols%7C_____1'
content = requests.get(main_url)
content.encoding = content.apparent_encoding

main=BeautifulSoup(content.text)

symbol_tags = main.find_all('a',attrs={'href':lambda x: x.startswith('../Symbols/')})
symbols_dict = {o.text:o['href'] for o in symbol_tags}

In [3]:
base = 'https://help.dyalog.com/18.2/Content/Language'

Card1:
+ Symbol : Name + Monadic Name + Dyadic Name
+ Symbol Name : Symbol
+ Monadic Name : Symbol
+ Dyadic Name : Symbol
+ Documentation Name : Symbol (only if different)
+ Documentation Name : Documentation
+ Symbol : Documentation
+ Documentation : Symbol
+ Documentation : Documentation Name

In [4]:
def get_hyperlink(url,url_type):
    return f" <br/><br/><a href='{url}'>Take me to {url_type}</a>"

In [5]:
cards = []
for i,sym in enumerate(L(*symbols_dict.keys())):
    doc_url = symbols_dict[sym]
    url1 = base + doc_url[2:]

    content = requests.get(url1)
    content.encoding = content.apparent_encoding
    doc_html = BeautifulSoup(content.text)

    doc_h5s = doc_html.find_all('h5')
    doc_h3 = doc_html.find('td').find('a',attrs={'name':lambda o: not o.startswith('kanchor')})
    
    
    if doc_html.find('td').text.strip() == '':
        _2 = doc_h3['name']
    else: _2 = doc_html.find('td').text.strip()
    
    # Name of glyph -> Symbol
    cards.append(pd.DataFrame([[_2 + "<br/><br/>What's the glyph look like?",
                                sym + get_hyperlink(url1,"symbol documentation"),
                               "Name->Sym"
                               ]]))
    urls = []
    for doc_h5 in doc_h5s:
        
        p = doc_h5.find_next_sibling('p')
        dy_or_mon_name = ''.join(re.findall('Dyadic|Monadic',doc_h5.text)) + ': ' + p.text.strip()
        
        # Mondadic/Dyadic Name -> Symbol
        cards.append(pd.DataFrame([[dy_or_mon_name + "<br/><br/>What's the glyph look like?",
                                    sym + get_hyperlink(url1,"symbol documentation"),
                                   "Name->Sym"
                                   ]]))
        _2  = _2 + ' <br/> ' + dy_or_mon_name 
        
        
        url_end = [o.find('a',attrs={'href':True}) for o in doc_h5.find_next_siblings('p') if o.find('a',attrs={'href':True}) is not None][0]['href']
#         url2 = base + p.find('a',attrs={'href':True})['href'][2:]
        url2 = base + url_end[2:]
        urls.append(url2)
        
        content = requests.get(url2)
        content.encoding = content.apparent_encoding
        doc_html = BeautifulSoup(content.text)
    
        try:
            desc = [o.text for o in doc_html.find('td').find_all_next('p') if o.text.strip() != '']

            documentation = ' <br/> '.join(desc)
            
            # Functional documentation -> Symbol
            cards.append(pd.DataFrame([[documentation + "<br/><br/>What's the glyph look like?",
                                        sym + get_hyperlink(url1,"symbol documentation") + get_hyperlink(url2,"primitive function documentation"),
                                       "Docs->Sym"
                                       ]]))
        except:
            print(i)
        try:
            doc_name = doc_html.find('td').find('a',attrs={'name':lambda o: not o.startswith('kanchor')})
            if doc_name['name'] not in _2:
                # Documentation Name -> Symbol
                cards.append(pd.DataFrame([[doc_name['name'] + "<br/><br/>What's this glyph look like?",
                                            sym + get_hyperlink(url1,"symbol documentation") + get_hyperlink(url2,"primitive function documentation"),
                                           "Name->Sym"
                                           ]]))

            # Documentation Name -> Functional Documentation
            # Functional Documentation -> Documentation Name
            cards.append(pd.DataFrame([
                    [doc_name['name'] + "<br/><br/>What's this glyph do?",
                     documentation + get_hyperlink(url1,"symbol documentation") + get_hyperlink(url2,"primitive function documentation"),
                     "Name->Docs"
                    ],
                    [documentation + "<br/><br/>What's the name of this glyph?",
                     doc_name['name'] + get_hyperlink(url1,"symbol documentation") + get_hyperlink(url2,"primitive function documentation"),
                    "Docs->Name"
                    ],
                               
                    ]))
        except:
            print(i)
    
    # Symbol -> Functional Documentation
    cards.append(pd.DataFrame([[sym + "<br/><br/>What's this glyph do?",
                                        documentation + ''.join([get_hyperlink(o,"Link") for o in urls]),
                                       "Sym->Docs"
                                       ]]))
    # Symbol -> Names
    cards.append(pd.DataFrame([[sym + "<br/><br/>What's this glyph called?  What's the Monadic/Dyadic names?",
                                _2 + get_hyperlink(url1,"symbol documentation"),
                               "Sym->Docs"]]))

65
72
72
72


In [6]:
''.join([get_hyperlink(o,"Link") for o in [url1,url2]])
# get_hyperlink(url1,"symbol documentation")
# get_hyperlink(url2,"primitive function documentation")

" <br/><br/><a href='https://help.dyalog.com/18.2/Content/Language/Symbols/Special Symbols.htm'>Take me to Link</a> <br/><br/><a href='https://help.dyalog.com/18.2/Content/Language/Defined Functions and Operators/TradFns/Model Syntax.htm'>Take me to Link</a>"

In [7]:
df = pd.concat(cards)
df.to_csv('DyalogDocs_AllCards.csv',index=False,header=False)
df.sample(15)

,0,1,2
0,The symbol chosen to represent Mix depends on ...,↑ <br/><br/><a href='https://help.dyalog.com/1...,Docs->Sym
0,Tally<br/><br/>What's this glyph do?,Y may be any array. R is a simple numeric sca...,Name->Docs
0,Classic Edition: the symbol ⌺ is not available...,⌺ <br/><br/><a href='https://help.dyalog.com/1...,Docs->Sym
0,Back Slash<br/><br/>What's the glyph look like?,\ <br/><br/><a href='https://help.dyalog.com/1...,Name->Sym
1,Y must be a simple character array of rank gre...,GradeDown(Dyadic) <br/><br/><a href='https://h...,Docs->Name
0,Unique<br/><br/>What's this glyph do?,Y may be any array. R is a vector of the uniqu...,Name->Docs
0,Beside<br/><br/>What's this glyph do?,g can be any monadic function which returns a ...,Name->Docs
0,-<br/><br/>What's this glyph do?,Y may be any numeric array. X may be any nume...,Sym->Docs
0,X and Y must be numeric arrays. X cannot be 1 ...,⍟ <br/><br/><a href='https://help.dyalog.com/1...,Docs->Sym
0,The form R←f⍀Y implies scan along the first ax...,⍀ <br/><br/><a href='https://help.dyalog.com/1...,Docs->Sym


# Stuff that doesn't work quite right
+ A few documentation pages have code cells mixed into the text.  Currently it only pulls \<p> text cells so the code is lost.  That could be fixed, but it's infrequent and not a huge deal since there's hyperlinks.
+ Fix the try/except stuff